In [4]:
import pandas as pd
import  os
import numpy as np
import psycopg2
import glob
import openpyxl

In [5]:
df = pd.read_excel("20210727_Chart of accounts.xlsx", sheet_name= "Hierarchy")
df.head()

,Level 6,Level 5,Level 4,Level 3,Level 2,Level 1,Level 0,Code
0,Profit & Loss,Income,Other Income,Other Income,Other Income,Other Income,Other Charges,223500001
1,Profit & Loss,Expenses,Operating expenses,Travel & Entertainments,Travel & Entertainments,Travel & Entertainments,Lodging,223400001
2,Balance Sheet,Assets,Current Assets,Cash and equivalent,Cash & Wallets,Cash & Wallets,Undeposited Funds,111000001
3,Balance Sheet,Assets,Current Assets,Cash and equivalent,Cash & Wallets,Cash & Wallets,Petty Cash,111000002
4,Balance Sheet,Assets,Current Assets,Cash and equivalent,Cash & Wallets,Cash & Wallets,MTN Mobile Money,111000003


In [6]:
file = "Hierarchy"
clean_tbl_name = file.lower().replace(" ","_")
clean_tbl_name

'hierarchy'

In [8]:
df.columns = [x.lower().replace(" ","_") for x in df.columns]
df.columns

Index(['level_6', 'level_5', 'level_4', 'level_3', 'level_2', 'level_1',
       'level_0', 'code'],
      dtype='object')

In [9]:
df.dtypes

level_6    object
level_5    object
level_4    object
level_3    object
level_2    object
level_1    object
level_0    object
code        int64
dtype: object

In [10]:
replacements = {
    'object' : 'varchar',
    'int64' : 'varchar'
}
replacements

{'object': 'varchar', 'int64': 'varchar'}

In [11]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'level_6 varchar, level_5 varchar, level_4 varchar, level_3 varchar, level_2 varchar, level_1 varchar, level_0 varchar, code varchar'

In [13]:
#open database connection


conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [15]:
#drop tables with same name
cursor.execute("drop table if exists Hierarchy;")

In [16]:
#creating tables
cursor.execute("create table hierarchy \
    (level_6 varchar, level_5 varchar, level_4 varchar, level_3 varchar, level_2 varchar, level_1 varchar, level_0 varchar, code varchar)" )

In [17]:
#insert values to tables
#save df to csv
df.to_csv('Hierarchy', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("Hierarchy")
print('file opened in memory')

file opened in memory


In [18]:
#upload to db
SQL_STATEMENT = """
COPY Hierarchy FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [19]:
cursor.execute("grant select on table hierarchy to public")
conn.commit()

cursor.close()
print('table hierarchy imported successfully')

table hierarchy imported successfully
